In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import Session
from snowflake.snowpark.types import StringType

session = get_active_session()


In [ ]:
from snowflake.snowpark import Session, DataFrame

def process_data(session: Session, month_date: str, event_date: str):
    delete_sql = f""" DELETE FROM TELKOMSEL_POC.CURATED.merge_revenue_ifrs_dd_stg_breakage WHERE event_date = '{event_date}' """

    session.sql(delete_sql).collect()

    # Step 2: Build the DataFrame with new data to append
    query = f"""
     SELECT a.trx_date,
           a.transaction_id,
           a.sku,
           a.business_id,
           a.rules_id,
           a.msisdn,
           a.brand,
           a.region_hlr,
           a.non_usage_flag,
           a.allowance,
           a.allowance_sub_type,
           a.allowances_descriptions,
           a.quota_name,
           a.bonus_apps_id,
           a.profile_name,
           a.cust_type_desc,
           a.cust_subtype_desc,
           a.customer_sub_segment,
           a.lacci_id,
           a.area_sales,
           a.region_sales,
           a.branch,
           a.subbranch,
           a.cluster_sales,
           a.provinsi,
           a.kabupaten,
           a.kecamatan,
           a.kelurahan,
           a.node_type,
           cast(sum(rev*seized_rate) AS decimal (38,2)) AS breakage,
           a.event_Date

    FROM (
    SELECT a.trx_date,
           a.transaction_id,
           a.sku,
           a.business_id,
           a.rules_id,
           a.msisdn,
           h.brand,
           h.region_hlr,
           a.non_usage_flag,
           CASE WHEN upper(a.allowance_sub_type) like '%DATA%'
                  OR upper(a.allowance_sub_type) like '%UPCC%' THEN 'DATA'
                WHEN upper(a.allowance_sub_type) like '%SMS%' THEN 'SMS'
                WHEN upper(a.allowance_sub_type) like '%VOICE%' THEN 'VOICE'
           ELSE 'NONUSAGE' END AS allowance,
           a.allowance_sub_type,
           a.allowances_descriptions,
           a.quota_name,
           a.bonus_apps_id,
           a.profile_name,
           h.cust_type_desc,
           h.cust_subtype_desc,
           h.customer_sub_segment,
           coalesce(b.lacci_id, c.lacci_id) AS lacci_id,
           coalesce(b.area_sales, c.area_sales) AS area_sales,
           coalesce(b.region_sales, c.region_sales) AS region_sales,
           coalesce(b.branch, c.branch) AS branch,
           coalesce(b.subbranch, c.subbranch) AS subbranch,
           coalesce(b.cluster_sales, c.cluster_sales) AS cluster_sales,
           coalesce(b.provinsi, c.provinsi) AS provinsi,
           coalesce(b.kabupaten, c.kabupaten) AS kabupaten,
           coalesce(b.kecamatan, c.kecamatan) AS kecamatan,
           coalesce(b.kelurahan, c.kelurahan) AS kelurahan,
           coalesce(b.node_type, c.node_type) AS node_type,
           sum(alloc_tp) AS rev,
           a.event_Date

    FROM (
    SELECT * FROM wisdom_poc_tokenized
    WHERE event_Date = '{event_date}'
      AND status = 'OK00'
    ) a

    LEFT JOIN (
    SELECT * FROM cb_profile_tnt_poc_tokenized
    WHERE event_Date = '{event_date}'
    ) h
    ON a.event_Date = h.event_Date
      AND a.msisdn = h.msisdn

    LEFT JOIN (
    SELECT a.* FROM (
    SELECT event_Date,
           msisdn,
           lacci_id,
           area_sales,
           region_sales,
           branch,
           subbranch,
           cluster_sales,
           provinsi,
           kabupaten,
           kecamatan,
           kelurahan,
           node_type,
           rank() OVER (PARTITION BY event_Date, msisdn ORDER BY lacci_id DESC) AS rnk
    FROM lacci_dom_ifrs_chg_mtd_poc_tokenized
    WHERE event_Date = '{event_date}'
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13
    ) a
    WHERE rnk = '1'
    ) b
    ON a.event_Date = b.event_Date
      AND a.msisdn = b.msisdn

    LEFT JOIN (
    SELECT a.* FROM (
    SELECT event_Date,
           msisdn,
           lacci_id,
           area_sales,
           region_sales,
           branch,
           subbranch,
           cluster_sales,
           provinsi,
           kabupaten,
           kecamatan,
           kelurahan,
           node_type,
           rank() OVER (PARTITION BY event_Date, msisdn ORDER BY lacci_id DESC) AS rnk
    FROM lacci_dom_ifrs_chg_mm_poc_tokenized
    WHERE event_date = '{month_date}'
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13
    ) a
    WHERE rnk = '1'
    ) c
    ON a.msisdn = c.msisdn
    
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31
    ) a

    INNER JOIN (
    SELECT * FROM breakage_reference_poc_tokenized
    WHERE event_date IN (SELECT MAX(event_date) FROM breakage_reference_poc_tokenized)
    ) b1
    ON a.allowance = b1.allowance_type
      AND a.region_sales = b1.region
    WHERE coalesce(lower(brand), 'prepaid') <> 'kartuhalo'

    GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31
    """

    df = session.sql(query)

    # Step 3: Append the query result to the target table
    df.write.mode('append').save_as_table('TELKOMSEL_POC.CURATEDmerge_revenue_ifrs_dd_stg_breakage')

    return f"Deleted and appended data for event_date: {event_date}"


In [ ]:
session.sproc.register(
    func=process_data,
    name="process_data_merge_revenue_ifrs_dd_stg_breakage",
    is_permanent=True,
    stage_location="@TSELPOC_STAGE",           # Replace with your actual stage
    replace=True,
    input_types=[StringType()],          # Instead of [str]
    return_type=StringType(),            # Instead of str or "STRING"
    packages=["snowflake-snowpark-python"]
)


In [ ]:
call process_data_merge_revenue_ifrs_dd_stg_breakage('2025-04-01','2025-04-02')